# Preprocess

Preprocess the raw results from OntoGPT

Draft: https://docs.google.com/document/d/1H103ux6Dd1_bPM0un4RwutBLcYJx-0ybil2AwlAvG_Q/edit#

## Initial setup

Import libraries, create a GO adapter (for calculating closures/ancestors)

In [1]:
# note the gpt4 dir includes combined results from davinci, 3.5 and 4
results_dir = "results/human/gpt4"

In [2]:
import yaml
from yaml import Loader
from collections import defaultdict

import pandas as pd
import math

from oaklib import get_adapter
from oaklib.datamodels.vocabulary import IS_A, PART_OF
from ontogpt.evaluation.enrichment.eval_enrichment import EvalEnrichment
go = get_adapter("sqlite:obo:go")

In [3]:
# Load the closure/ancestor map
# (takes a minute)
closure_map = defaultdict(set)
for s, _, o in go.relationships(predicates=[IS_A, PART_OF], include_entailed=True):
    closure_map[s].add(o)

In [4]:
print(len(closure_map))

84220


In [5]:
# ruamel is faster than pyyaml
from ruamel.yaml import YAML
ryaml = YAML()

## Load YAML results

See OntoGPT for details of data model

These results are nested, with a parent gene set object containing all runs.
We want to flatten this for analysis

In [6]:
from ontogpt.evaluation.enrichment.eval_enrichment import GeneSetComparison

In [7]:
# assumes comparisons have been run and concatenated (see Makefile) 
import glob
def load_gene_set_results():
    results = []
    for fn in glob.glob(f"{results_dir}/*.yaml"):
        print(fn)
        with open(fn) as f:
            #obj = yaml.load(f, Loader)
            #obj = yaml.safe_load(f)
            obj = ryaml.load(f)
            results.extend(obj)
    return results

In [ ]:
comps = load_gene_set_results()

results/human/gpt4/canonical-glycolysis-gocam-results-2.yaml
results/human/gpt4/bicluster_RNAseqDB_1001-results-2.yaml
results/human/gpt4/HALLMARK_HYPOXIA-results-2.yaml
results/human/gpt4/HALLMARK_DNA_REPAIR-results-2.yaml
results/human/gpt4/HALLMARK_G2M_CHECKPOINT-results-2.yaml
results/human/gpt4/EDS-results-2.yaml
results/human/gpt4/HALLMARK_IL2_STAT5_SIGNALING-results-2.yaml
results/human/gpt4/HALLMARK_EPITHELIAL_MESENCHYMAL_TRANSITION-results-2.yaml
results/human/gpt4/HALLMARK_PI3K_AKT_MTOR_SIGNALING-results-2.yaml
results/human/gpt4/HALLMARK_COAGULATION-results-2.yaml
results/human/gpt4/peroxisome-results-2.yaml
results/human/gpt4/HALLMARK_APICAL_JUNCTION-results-2.yaml
results/human/gpt4/HALLMARK_ANGIOGENESIS-results-2.yaml
results/human/gpt4/go-postsynapse-calcium-results-2.yaml
results/human/gpt4/HALLMARK_BILE_ACID_METABOLISM-results-2.yaml
results/human/gpt4/HALLMARK_CHOLESTEROL_HOMEOSTASIS-results-2.yaml
results/human/gpt4/bicluster_RNAseqDB_1002-results-2.yaml
results/huma

In [ ]:
len(comps)

In [ ]:
results_by_gene_sets = comps

## Create the main data frame

For each gene set, we will create a list of rows,
for each combination of method + parameters, plus a cutoff.

A cutoff of zero means we only consider the *top rank* GO term from enrichment.
If the results contain this, it's a true positive; otherwise we count as a single false positive.

Otherwise we look at all enrichment results with p-val less than the cutoff. If predicted terms match these,
it's a true positive.

For calculating false negatives, we check both ancestors and descendants

In [ ]:
import itertools
pairs = list(itertools.product([0.005, 0.05, 99], [False, True], [1, 5, 10, 25, 5000]))

def eval_gene_set_result(gs):
    """
    For each gene set makes, sets of rows, for each combination of method/params,
    plus each combination of p-value cutoff
    """
    rows = []
    std = gs["payloads"]["standard"]
    if "enrichment_results" not in std:
        print(f"NO GOLD STANDARD: {gs['name']}")
    expected_results = std.get("enrichment_results", [])
    for run, method_result in gs["payloads"].items():
        predicted_term_ids = method_result.get("term_ids", [])

        #closure_map = defaultdict(list)
        predicted_term_closure_ids = set()
        for t in predicted_term_ids:
            predicted_term_closure_ids.update(closure_map.get(t, set()))
        #for s, _, o in go.relationships(subjects=predicted_term_ids, predicates=[IS_A, PART_OF], include_entailed=True):
            #closure_map[s].append(o)
        #    predicted_term_closure_ids.add(o)
        for cutoff, use_closure, top_n in pairs:
            #if top_n == 1 and cutoff != 0.05:
            #    continue
            row = {}

            ###previously this was from:
            ##go_term_ids = [t for t in payload.term_ids if t.startswith("GO:")]
            row['go_term_ids'] = predicted_term_closure_ids#gs["payloads"]["closure"]["term_ids"]#go_term_ids
            row["name"] = f"{gs['name']}-{cutoff}"
            row["cutoff"] = cutoff
            row["closure"] = use_closure
            row["top_n"] = top_n
            method = method_result.get("method", "")
            approach = "gpt"
            if method == "no_synopsis":
                src = "NONE"
            elif method == "ontological_synopsis":
                src = "GO"
            elif method == "narrative_synopsis":
                src = "RefSeq"
            else:
                approach = method
                src = ""
            row["source"] = src
            model = method_result.get("model", "")
            if model == "gpt-4":
                model = "4"
            elif model == "gpt-3.5-turbo":
                model = "3.5"
            elif model == "text-davinci-003":
                model = "3"
            row["model"] = model
            row["method"] = approach
            if model:
                row["method_desc"] = f"{method}-{model}"
            else:
                row["method_desc"] = method_result.get("truncation_factor", "")
            row["run"] = run
            for k in ["truncation_factor", "prompt_variant", "response_token_length"]:
                row[k] = method_result.get(k, "")
            row["prompt_length"] = len(method_result.get("prompt", ""))
            true_positive_terms = []
            false_negative_terms = []
            more_specific_false_negative_terms = []   # predicted a descendant
            more_general_false_negative_terms = []   # predicted a descendant
            unparsed_terms = []
            standard_enrichment_results = [(r["p_value_adjusted"], r["class_id"]) for r in expected_results]
            enrichment_closure = set()
            enrichment_term_ids = set()
            if cutoff == 99:
                # extend the enrichment results but with max cutoff
                for x in gs["payloads"]["closure"]["term_ids"]:
                    standard_enrichment_results.append((cutoff, x))
            visited_terms = set()
            n = 0
            for p_val, true_term_id in standard_enrichment_results:
                if cutoff > 0 and p_val > cutoff:
                    break
                if true_term_id in visited_terms:
                    continue
                n += 1
                if n > top_n:
                    break
                enrichment_closure.update(closure_map.get(true_term_id, set()))
                enrichment_term_ids.add(true_term_id)
                true_term_closure_ids = closure_map.get(true_term_id, set())
                if use_closure:
                    visited_terms.update(true_term_closure_ids)
                else:
                    visited_terms.add(true_term_id)
                if true_term_id in predicted_term_ids:
                    true_positive_terms.append(true_term_id)
                elif true_term_id in predicted_term_closure_ids:
                    # predicted a more specific term
                    if use_closure:
                        true_positive_terms.append(true_term_id)
                    else:
                        false_negative_terms.append(true_term_id)
                    more_specific_false_negative_terms.append(true_term_id)
                elif true_term_closure_ids.intersection(predicted_term_ids):
                    # predicted a more general term
                    if use_closure:
                        true_positive_terms.append(true_term_id)
                    else:
                        false_negative_terms.append(true_term_id)
                    more_general_false_negative_terms.append(true_term_id)
                else:
                    false_negative_terms.append(true_term_id)
                if cutoff == 0:
                    break
            false_positive_terms = []
            if top_n > 1:
                for t in predicted_term_ids:
                    if t not in true_positive_terms:
                        if t.startswith("GO:"):
                            if not use_closure:
                                false_positive_terms.append(t)
                            else:
                                if t in enrichment_closure:
                                    # prediction is more general
                                    pass
                                elif closure_map.get(t, set()).intersection(enrichment_term_ids):
                                    # prediction is more specific
                                    pass
                                else:
                                    false_positive_terms.append(t)
                        elif ":" in t:
                            # MONDO, UBERON, etc
                            pass
                        else:
                            unparsed_terms.append(t)
            else:
                if not true_positive_terms:
                    if predicted_term_ids:
                        false_positive_terms.append(predicted_term_ids[0])
            row["true_positives"] = len(true_positive_terms)
            row["false_positives"] = len(false_positive_terms)
            row["false_negatives"] = len(false_negative_terms)
            row["more_general_false_negatives"] = len(more_general_false_negative_terms)
            row["more_specific_false_negatives"] = len(more_specific_false_negative_terms)
            row["all_predictions_closure"] = len(predicted_term_closure_ids)
            row["unparsed"] = len(unparsed_terms)
            row["true_positive_terms"] = "|".join(true_positive_terms)
            row["false_positive_terms"] = "|".join(false_positive_terms)
            row["false_negative_terms_example20"] = "|".join(false_negative_terms)[0:20]
            row["unparsed_terms"] = "|".join(unparsed_terms)
            row["gene_set_size"] = len(gs.get("gene_symbols"))
            
            denom1 = row["true_positives"] + row["false_positives"]
            if denom1 != 0:
                row["precision"] = row["true_positives"] / denom1
            else:
                row["precision"] = 0

            denom2 = row["true_positives"] + row["false_negatives"]
            if denom2 != 0:
                row["recall"] = row["true_positives"] / denom2
            else:
                row["recall"] = 0
                
            denom3 = row["true_positives"] + row["more_general_false_negatives"]
            if denom3 != 0:
                row["recall_general"] = row["true_positives"] / denom3
            else:
                row["recall_general"] = 0
                
            denom4 = row["true_positives"] + row["more_specific_false_negatives"]
            if denom4 != 0:
                row["recall_specific"] = row["true_positives"] / denom4
            else:
                row["recall_specific"] = 0
            
            f1_denom = row["true_positives"] + 0.5* (row["false_positives"] + row["false_negatives"]) 
            if(f1_denom != 0):
                row["f1_score"] = row["true_positives"] / (f1_denom)
            else:
                row["f1_score"] = None

            #mcc_denom = math.sqrt((row["true_positives"] + row["false_positives"]) * (row["true_positives"] + row["false_negatives"]) * (row["true_negatives"] + row["false_positives"]) * (row["true_negatives"] + row["false_negatives"]))
            #mcc_num = (row["true_negatives"] * row["true_positives"]) -  (row["false_negatives"] * row["false_positives"])
            #if(f1_denom != 0):
            #    row["mcc"] =  mcc_num / (mcc_denom)
            #else:
            #    row["mcc"] = None

            #row["gene_symbols"] = "|".join(gs.get("gene_symbols", []))
            #row["gene_ids"] = "|".join(gs.get("gene_ids", []))
            rows.append(row)
    return rows

rows = eval_gene_set_result(results_by_gene_sets[4])
df = pd.DataFrame(rows)
df.to_csv("results/TEST-processed.tsv", sep="\t", index=False)
df

In [ ]:
rows = []
for gs in results_by_gene_sets:
    this_rows = eval_gene_set_result(gs)
    rows.extend(this_rows)
    print(len(rows))
    df = pd.DataFrame(rows)
    df.to_csv("results/TEMP.tsv", sep="\t", index=False)
    
    #if len(rows) % 10 == 0:
    #    print(f"Processed {len(rows)})
              
df = pd.DataFrame(rows)
pd.set_option('display.max_rows', 10)
df.to_csv("results/processed.tsv", sep="\t", index=False)
df

In [ ]:
print(df.shape)
#df[[model, method]].drop_duplicates()
print(df.shape)

In [ ]:
df["method"].unique()